## Load Dataset

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
# libgomp issue, must import n2 before torch
from n2 import HnswIndex

In [4]:
import sys

sys.path.insert(0, '..')

In [5]:
import os
home_dir = os.getenv('HOME')

https://dbs.uni-leipzig.de/research/projects/object_matching/benchmark_datasets_for_entity_resolution

https://www.informatik.uni-leipzig.de/~saeedi/NCVoters_Readme.txt

```
5 party:
--------- 
5 sources. 
Each source 1000,000 entities.
There is one file per source, so totally 5 files
****************************************************
****************************************************
****************************************************
Fields:
---------- 
recId: entites with the same recId refer to the same entity.
givenname: 
surname: 
post code: 
suburb:
```

In [6]:
import glob
import csv
from tqdm.auto import tqdm

current_row_id = 0
row_dict = {}
rows_total = 5000000
cluster_attr = 'recid'

with tqdm(total=rows_total) as pbar:
    for filename in glob.glob(f'{home_dir}/Downloads/5Party-ocp20/*.csv'):
        with open(filename) as f:
            for row in csv.DictReader(f):
                row['id'] = current_row_id
                row[cluster_attr] = int(row[cluster_attr])  # convert cluster_attr to int
                row_dict[current_row_id] = row
                current_row_id += 1
                pbar.update(1)

In [7]:
row_dict[0]

{'recid': 7852009,
 'givenname': 'kadelyn',
 'surname': 'gragnani',
 'suburb': 'waxhaw',
 'postcode': '28|73',
 'id': 0}

## Preprocess

In [8]:
attr_list = ['givenname', 'surname', 'suburb', 'postcode']

In [9]:
import unidecode

def clean_str(s):
    return unidecode.unidecode(s).lower().strip()[:30]

for row in tqdm(row_dict.values()):
    for attr in attr_list:
        row[attr] = clean_str(row[attr])

## Init Data Module

In [10]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [11]:
alphabet = list('0123456789abcdefghijklmnopqrstuvwxyz!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ ')

In [12]:
attr_info_dict = {
    'givenname': {
        'is_multitoken': False,
        'tokenizer': None,
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'surname': {
        'is_multitoken': False,
        'tokenizer': None,
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'suburb': {
        'is_multitoken': False,
        'tokenizer': None,
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'postcode': {
        'is_multitoken': False,
        'tokenizer': None,
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    }
}

In [13]:
from entity_embed import build_row_encoder

row_encoder = build_row_encoder(attr_info_dict, row_dict=row_dict)
row_encoder.attr_info_dict

23:09:15 INFO:For attr='givenname', computing actual alphabet and max_str_len
23:09:18 INFO:For attr='givenname', using actual_max_str_len=16
23:09:18 INFO:For attr='surname', computing actual alphabet and max_str_len
23:09:21 INFO:actual_max_str_len=21 must be pair to enable NN pooling. Updating to 22
23:09:22 INFO:For attr='surname', using actual_max_str_len=22
23:09:22 INFO:For attr='suburb', computing actual alphabet and max_str_len
23:09:25 INFO:actual_max_str_len=21 must be pair to enable NN pooling. Updating to 22
23:09:25 INFO:For attr='suburb', using actual_max_str_len=22
23:09:25 INFO:For attr='postcode', computing actual alphabet and max_str_len
23:09:27 INFO:actual_max_str_len=9 must be pair to enable NN pooling. Updating to 10
23:09:27 INFO:For attr='postcode', using actual_max_str_len=10


{'givenname': OneHotEncodingInfo(is_multitoken=False, tokenizer=None, alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' '], max_str_len=16),
 'surname': OneHotEncodingInfo(is_multitoken=False, tokenizer=None, alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' '], max_str_len=22),
 'suburb': OneHotEncodingInfo(is_multitoken=False, tokenizer=None, alphabet=['0', '1', '2', '3', '4', '5', '6', '7

In [14]:
from entity_embed import EntityEmbedDataModule

train_cluster_len = 1000
valid_cluster_len = 1000
test_cluster_len = 200_000
datamodule = EntityEmbedDataModule(
    row_dict=row_dict,
    cluster_attr=cluster_attr,
    row_encoder=row_encoder,
    pos_pair_batch_size=45,
    neg_pair_batch_size=1225,
    row_batch_size=64,
    train_cluster_len=train_cluster_len,
    valid_cluster_len=valid_cluster_len,
    test_cluster_len=test_cluster_len,
    only_plural_clusters=True,
    log_empty_vals=False,
    random_seed=random_seed
)

## Training

In [15]:
tb_log_dir = 'tb_logs'

In [16]:
# %load_ext tensorboard
# %tensorboard --logdir tb_logs

In [17]:
from entity_embed import LitEntityEmbed

model = LitEntityEmbed(
    datamodule,
    ann_k=10,
    sim_threshold=0.5,
)

In [18]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

max_epochs = 50
early_stop_callback = EarlyStopping(
   monitor='valid_recall',
   min_delta=0.00,
   patience=10,
   verbose=True,
   mode='max'
)
trainer = pl.Trainer(
    gpus=1,
    max_epochs=max_epochs,
    check_val_every_n_epoch=1,
    callbacks=[early_stop_callback],
    logger=TensorBoardLogger(tb_log_dir, name='voters')
)

23:09:28 INFO:GPU available: True, used: True
23:09:28 INFO:TPU available: None, using: 0 TPU cores
23:09:28 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [19]:
trainer.fit(model, datamodule)

23:09:40 WARNING:train_len + valid_len + test_len=202000 is less than len(all_cluster_id_set)=500000
23:09:42 INFO:Train pair count: 6679
23:09:42 INFO:Valid pair count: 6645
23:09:42 INFO:Test pair count: 1331364
23:09:45 INFO:
  | Name        | Type           | Params
-----------------------------------------------
0 | blocker_net | BlockerNet     | 2.5 M 
1 | losser      | NTXentLoss     | 0     
2 | miner       | BatchHardMiner | 0     
-----------------------------------------------
2.5 M     Trainable params
0         Non-trainable params
2.5 M     Total params


1

In [20]:
model._get_signature_weights()

[('givenname', tensor(0.2381)),
 ('surname', tensor(0.3102)),
 ('suburb', tensor(0.2303)),
 ('postcode', tensor(0.2214))]

In [21]:
import gc
del trainer
gc.collect()

84

## Testing manually 

In [22]:
datamodule.setup(stage='test')

23:10:51 WARNING:train_len + valid_len + test_len=202000 is less than len(all_cluster_id_set)=500000
23:10:52 INFO:Train pair count: 6679
23:10:52 INFO:Valid pair count: 6645
23:10:52 INFO:Test pair count: 1331364


In [23]:
test_row_dict = datamodule.test_row_dict
test_vector_dict = model.predict(
    row_dict=test_row_dict,
    row_encoder=row_encoder,
    batch_size=16
)

In [24]:
embedding_size = model.blocker_net.embedding_size
test_true_pair_set = datamodule.test_true_pair_set

In [25]:
import gc
del model
del datamodule
gc.collect()

927

In [26]:
assert len(test_vector_dict) == len(test_row_dict)

In [27]:
%%time

from entity_embed import ANNEntityIndex

ann_index = ANNEntityIndex(embedding_size=embedding_size)
ann_index.insert_vector_dict(test_vector_dict)
ann_index.build()

CPU times: user 15min 49s, sys: 1.62 s, total: 15min 51s
Wall time: 1min 28s


In [28]:
%%time

ntop = 10
sim_threshold = 0.5

found_pair_set = ann_index.search_pairs(
    k=ntop,
    sim_threshold=sim_threshold
)

CPU times: user 21min 46s, sys: 379 ms, total: 21min 47s
Wall time: 1min 56s


In [29]:
from entity_embed.evaluation import pair_entity_ratio

pair_entity_ratio(len(found_pair_set), len(test_row_dict))

6.1549297184483756

In [30]:
from entity_embed.evaluation import precision_and_recall

precision_and_recall(found_pair_set, test_true_pair_set)

(0.2705316779105797, 0.9995568454607455)

In [31]:
false_positives = list(found_pair_set - test_true_pair_set)
len(false_positives)

3588332

In [32]:
false_negatives = list(test_true_pair_set - found_pair_set)
len(false_negatives)

590

In [33]:
cos_similarity = lambda a, b: np.dot(a, b)

In [34]:
for (id_left, id_right) in false_negatives[:10]:
    display(
        (
            cos_similarity(test_vector_dict[id_left], test_vector_dict[id_right]),
            row_dict[id_left], row_dict[id_right]
        )
    )

(0.5087182,
 {'recid': 3259167,
  'givenname': 'lindsay',
  'surname': 'xmeltzer',
  'suburb': 'salisbury',
  'postcode': '2814q',
  'id': 4155},
 {'recid': 3259167,
  'givenname': 'lindsay',
  'surname': 'schmeltzer',
  'suburb': 'salisbury',
  'postcode': '28144',
  'id': 1742878})

(0.603132,
 {'recid': 932111,
  'givenname': 'robert',
  'surname': 'fillips',
  'suburb': 'duhn',
  'postcode': '28334',
  'id': 3063662},
 {'recid': 932111,
  'givenname': 'robert',
  'surname': 'phillips',
  'suburb': 'dunn',
  'postcode': '28334',
  'id': 4131364})

(0.6904638,
 {'recid': 5584048,
  'givenname': 'angel',
  'surname': 'jones',
  'suburb': 'roanocke rapids',
  'postcode': '2787o',
  'id': 1042238},
 {'recid': 5584048,
  'givenname': 'ankel',
  'surname': 'jories',
  'suburb': 'roanoke rapids',
  'postcode': '27870',
  'id': 4042346})

(0.8965926,
 {'recid': 3789474,
  'givenname': 'dorothy',
  'surname': 'phillips',
  'suburb': 'advavce',
  'postcode': '27004',
  'id': 1036494},
 {'recid': 3789474,
  'givenname': 'doroty',
  'surname': 'phillips',
  'suburb': 'advance',
  'postcode': '27056',
  'id': 4037214})

(0.6543062,
 {'recid': 4207550,
  'givenname': 'tikia',
  'surname': 'watkins',
  'suburb': 'greensboro',
  'postcode': '27406',
  'id': 546941},
 {'recid': 4207550,
  'givenname': 'tikia',
  'surname': 'atkins',
  'suburb': 'greensboro',
  'postcode': '27806',
  'id': 4036883})

(0.59911966,
 {'recid': 584874,
  'givenname': 'james',
  'surname': 'mcrae',
  'suburb': 'laurinburg',
  'postcode': '28352',
  'id': 853639},
 {'recid': 584874,
  'givenname': 'jamws',
  'surname': 'iiicrae',
  'suburb': 'laurinburg',
  'postcode': '28352',
  'id': 1016987})

(0.56667227,
 {'recid': 5601884,
  'givenname': 'maria',
  'surname': 'chaiiiba',
  'suburb': 'charlotte',
  'postcode': '28527',
  'id': 1031015},
 {'recid': 5601884,
  'givenname': 'maria',
  'surname': 'xhamba',
  'suburb': 'charlotte',
  'postcode': '282z7',
  'id': 2031014})

(0.9284033,
 {'recid': 3380075,
  'givenname': 'sacqueline',
  'surname': 'hamilton',
  'suburb': 'rutherfordton',
  'postcode': '28539',
  'id': 1073481},
 {'recid': 3380075,
  'givenname': 'jacquelin',
  'surname': 'hamilton',
  'suburb': 'rutherfordton',
  'postcode': '28189',
  'id': 4073494})

(0.6297718,
 {'recid': 4844175,
  'givenname': 'aahlene',
  'surname': 'rorke',
  'suburb': 'raleigh',
  'postcode': '27615',
  'id': 3016080},
 {'recid': 4844175,
  'givenname': 'arlene',
  'surname': 'roure',
  'suburb': 'raleign',
  'postcode': '27615',
  'id': 4016029})

(0.58275825,
 {'recid': 7510874,
  'givenname': 'axley',
  'surname': 'tompson',
  'suburb': 'stanley',
  'postcode': '28164',
  'id': 36955},
 {'recid': 7510874,
  'givenname': 'ashley',
  'surname': 'thompson',
  'suburb': 'stanley',
  'postcode': '28164',
  'id': 2642636})